# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [ ]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=0"
#URL = "http://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=jobsite&salary=%2420%2C000%2B&radius=25&l=New+York&fromage=any&limit=50&sort=&psf=advsrch"

In [ ]:
import requests
import bs4
import urllib2
import numpy as np
import pandas as pd
#import seaborn as sns
from bs4 import BeautifulSoup

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

import statsmodels.api as sm

In [ ]:
## YOUR CODE HERE

title=[]
company=[]
location=[]
salary=[]
summary=[]

def scraper(URL,city):
    dom = BeautifulSoup(requests.get(URL).text,"lxml")

    jobDivs = dom.find_all("div", {"class":"result","class":"row"})

    for div in jobDivs:
        divSoup = BeautifulSoup(unicode.join(u'\n',map(unicode,div)),"lxml")

        #get title
        title.append(divSoup.find("a",{"data-tn-element":"jobTitle"})['title'])

        #get company
        company1 = divSoup.find("a",{"data-tn-element":"companyName"})
        company2 = divSoup.find("span",{"class":"company"})
        company3 = divSoup.find("span",{"itemprop":"name"})
        if(company1 != None):
            #print "1: ",company1.contents[-1].strip()
            company.append(company1.contents[-1].strip())
        elif(company2 != None):
            #print "2: ",company2.strip()
            company.append(company2.text.strip())
        elif(company3 != None):
            if(company3.string != None):
                #print "3a: ",company3.string.strip()
                company.append(company3.string.strip())
            else:
                #print "3b: ",company3.contents[-1].string.strip()
                if(company3.contents[-1].string != None):
                    company.append(company3.contents[-1].string.strip())
                else:
                    company.append("What:" + company3.contents[-1].text)
                    
        else:
            print "N/A"

        #get location 
        #locationString = divSoup.find("span",{"class":"location"}).string
        #if (locationString == None):
        #    locationString = divSoup.find("span",{"itemprop":"addressLocality"}).contents[0].strip()
        location.append(city)

        #get salary

        salaryItem = divSoup.find("span",{"class":"location"}).findNextSibling("div")
        salaryItem2 = divSoup.find("td",{"class":"snip"}).findChild('nobr')
        if (salaryItem != None):
            salary.append(salaryItem.text)
        elif(salaryItem2 != None):
            #print "nobr: ", salaryItem2.string
            salary.append(salaryItem2.string)
        else:
            salary.append(np.nan)

        #get summary

        #print divSoup.find("span",{"class":"summary"}).text
        #summary.append(divSoup.find("span",{"class":"summary"}).text)



Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [ ]:
max_results_per_city = 1000

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', "Atlanta","Seattle","Los+Angeles","Washington%2C+DC","Huntsville",
                 "San+Diego","Austin","Denver","St.+Louis","Boston","Portland","Philadelphia","Dallas","Houston","San+Antonio","Charlotte"]):
    print city
    for start in range(0, max_results_per_city, 10):
        print "Results: ", start
        url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l="+ city +"&start=" + str(start)
        scraper(url_template, city)

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [ ]:
print len(title)
print len(salary)
print len(location)
print len(salary)

In [ ]:
length = len(title)-1

In [ ]:
df = pd.DataFrame({'title':title[0:length], 'company':company[0:length], 'location':location[0:length], 'salary':salary[0:length]})

In [ ]:
df.describe()

In [ ]:
df = df.dropna()
df

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [ ]:
yearly = pd.DataFrame(columns=('company', 'location', 'salary', 'title'))
for index, row in df.iterrows():
    if "a year" in row['salary']:
        yearly.loc[index] = row
#yearly

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [ ]:
import re

for index, row in yearly.iterrows():
    if "-" in row['salary']:
        tokenized = row['salary'].split("-")
        #print "First: ",re.sub("[^0-9]", "", tokenized[0].strip("$")).strip()
        #print "Second: ",re.sub("[^0-9]", "", tokenized[1].strip("$")).strip()
        lower = int(re.sub("[^0-9]", "", tokenized[0]).strip())
        higher = int(re.sub("[^0-9]", "", tokenized[1]).strip())
        average = (higher + lower)/2
        yearly.loc[index][2] = average
    else:
        yearly.loc[index][2] = int(re.sub("[^0-9]", "", row['salary']).strip())
        
    yearly.loc[index][1] = re.sub("\d+","", row['location']).strip() 
    
yearly.head()

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

yearly['salary'] = yearly['salary'].astype(int)
#print_full(yearly['salary'])

### Save your results as a CSV

In [ ]:
yearly.describe()

yearly.to_csv("../assets/indeed_salary_data",encoding='utf-8',index=False)

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [61]:
import requests
import bs4
import urllib2
import numpy as np
import pandas as pd
#import seaborn as sns
from bs4 import BeautifulSoup

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import sklearn.metrics as metrics

import statsmodels.api as sm

In [62]:
data = pd.read_csv('../assets/indeed_salary_data')

In [63]:
data.head()

,company,location,salary,title
0,Texas A&M University,Houston,70000,Bioinformatics Postdoctoral Fellow
1,Baylor College of Medicine,Houston,86500,Senior Staff Scientist / Technical Director
2,Oscar Technology,Houston,160000,Big Data Analytics Manager
3,Baylor College of Medicine,Houston,93431,Java Programmer
4,Baylor College of Medicine,Houston,67632,Laboratory Manager


In [64]:
data.count()

company     1318
location    1318
salary      1318
title       1318
dtype: int64

In [65]:
data = data.drop_duplicates()
data.count()

company     424
location    424
salary      424
title       424
dtype: int64

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [66]:
meanSalary = np.mean(data["salary"])

def HigherOrLower(salary):
    if salary > meanSalary:
        return 1
    else:
        return 0

data["HighLow"] = map(HigherOrLower,data["salary"])
#smdata = data
#smdata["Intercept"] = 1
data["Intercept"] = 1

#### Thought experiment: What is the baseline accuracy for this model?

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [67]:
def categorizeTitles(string):
    title = string.lower()
    if("data scientist" in title):
        return 0
    elif("statistician" in title):
        return 1
    elif("data analyst" in title):
        return 2
    elif("data engineer" in title):
        return 3
    elif("research analyst" in title):
        return 4
    elif("software" in title or "engineer" in title or "developer" in title):
        return 5
    elif("analyst" in title):
        return 6
    elif("director" in title):
        return 7
    elif("manager" in title):
        return 8
    elif("scientist" in title):
        return 9
    else:
        return 10
    
categorizedTitles = map(categorizeTitles,data.title)
data["categorizedTitle"] = categorizedTitles
#smdata["categorizedTitle"] = categorizedTitles

In [68]:
def categorizeLevel(string):
    title = string.lower()
    if("junior" in title or "jr." in title):
        return 0
    elif("mid-level" in title):
        return 2
    elif("senior" in title or "sr." in title):
        return 1
    elif("principal" in title):
        return 2
    elif("lead" in title):
        return 3
    else:
        return 4
    
categorizedLevels = map(categorizeLevel,data.title)
data["categorizedLevel"] = categorizedLevels

In [69]:
locationData = data.groupby("location",as_index="False").count()
mostCommonLocation = locationData.idxmax()[1]
print "Most Common Location: ", mostCommonLocation

companyData = data.groupby("company",as_index="False").count()
mostCommonCompany = companyData.idxmax()[1]
print "Most Common Company: ", mostCommonCompany

titleData = data.groupby("categorizedTitle",as_index="False").count()
mostCommonTitle = titleData.idxmax()[1]
print "Most Common Categorized Title: ", mostCommonTitle

Most Common Location:  Boston
Most Common Company:  Workbridge Associates
Most Common Categorized Title:  0


In [70]:
categories = ["location"]
for category in categories:
    series = data[category]
    dummies = pd.get_dummies(series, prefix=category)
    data = pd.concat([data, dummies], axis=1)
    #if(categories != "company"):
        #smdata = pd.concat([smdata, dummies], axis = 1)
    
#delete most common location for colinearity
del data["location_"+mostCommonLocation]
#del data["company_"+mostCommonCompany]
#del data["categorizedTitle_"+str(mostCommonTitle)]

#del smdata["location_"+mostCommonLocation]
#del smdata["company_"+mostCommonCompany]
#del smdata["categorizedTitle_"+str(mostCommonTitle)]

#smdata

In [71]:
data.head()

,company,location,salary,title,HighLow,Intercept,categorizedTitle,categorizedLevel,location_Atlanta,location_Austin,...,location_Los+Angeles,location_New+York,location_Philadelphia,location_Portland,location_San+Antonio,location_San+Diego,location_San+Francisco,location_Seattle,location_St.+Louis,location_Washington%C+DC
0,Texas A&M University,Houston,70000,Bioinformatics Postdoctoral Fellow,0,1,10,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baylor College of Medicine,Houston,86500,Senior Staff Scientist / Technical Director,0,1,7,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Oscar Technology,Houston,160000,Big Data Analytics Manager,1,1,8,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Baylor College of Medicine,Houston,93431,Java Programmer,0,1,10,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Baylor College of Medicine,Houston,67632,Laboratory Manager,0,1,8,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
X = data.iloc[:,8:]
y = data["HighLow"]
y.name = "Actual"

In [75]:
logit = sm.Logit(y, X)
result = logit.fit()
result.summary()

         Current function value: 0.595938
         Iterations: 35


F:\Programs\Anaconda\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Actual   No. Observations:                  424
Model:                          Logit   Df Residuals:                      406
Method:                           MLE   Df Model:                           17
Date:                Fri, 01 Jul 2016   Pseudo R-squ.:                  0.1375
Time:                        06:41:44   Log-Likelihood:                -252.68
converged:                      False   LL-Null:                       -292.97
                                        LLR p-value:                 3.021e-10
============================================================================================
                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
location_Atlanta            -0.8473      0.488     -1.736      0.082        -1.804     0.109
location_Austin                   0      0.707          0      1.000        -1.386     1.386
location_Charlotte          20.5444   1.45e+04      0.001      0.999     -2.83e+04  2.84e+04
location_Chicago             0.1542      0.321      0.480      0.631        -0.475     0.784
location_Dallas             -0.6931      0.612     -1.132      0.258        -1.893     0.507
location_Denver             -1.3218      0.563     -2.349      0.019        -2.425    -0.219
location_Houston            -2.3026      1.049     -2.195      0.028        -4.358    -0.247
location_Huntsville         18.3274   9544.344      0.002      0.998     -1.87e+04  1.87e+04
location_Los+Angeles        -0.5306      0.399     -1.331      0.183        -1.312     0.250
location_New+York                 0      0.289          0      1.000        -0.566     0.566
location_Philadelphia        0.1542      0.556      0.277      0.782        -0.936     1.245
location_Portland          -20.5444   1.45e+04     -0.001      0.999     -2.84e+04  2.83e+04
location_San+Antonio       -20.5444   1.45e+04     -0.001      0.999     -2.84e+04  2.83e+04
location_San+Diego          -0.6931      0.612     -1.132      0.258        -1.893     0.507
location_San+Francisco       0.9163      0.483      1.897      0.058        -0.030     1.863
location_Seattle             2.0541      0.475      4.324      0.000         1.123     2.985
location_St.+Louis          -1.3218      0.563     -2.349      0.019        -2.425    -0.219
location_Washington%C+DC     0.1603      0.284      0.565      0.572        -0.396     0.716
============================================================================================
"""

In [76]:
result = logit.fit_regularized()
result.summary()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.595937655792
            Iterations: 104
            Function evaluations: 105
            Gradient evaluations: 104


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Actual   No. Observations:                  424
Model:                          Logit   Df Residuals:                      406
Method:                           MLE   Df Model:                           17
Date:                Fri, 01 Jul 2016   Pseudo R-squ.:                  0.1375
Time:                        06:41:48   Log-Likelihood:                -252.68
converged:                       True   LL-Null:                       -292.97
                                        LLR p-value:                 3.021e-10
============================================================================================
                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
location_Atlanta            -0.8473      0.488     -1.736      0.082        -1.804     0.109
location_Austin           1.648e-16      0.707   2.33e-16      1.000        -1.386     1.386
location_Charlotte          34.5689   1.68e+07   2.06e-06      1.000     -3.29e+07  3.29e+07
location_Chicago             0.1541      0.321      0.480      0.631        -0.475     0.784
location_Dallas             -0.6931      0.612     -1.132      0.258        -1.893     0.507
location_Denver             -1.3218      0.563     -2.349      0.019        -2.425    -0.219
location_Houston            -2.3026      1.049     -2.195      0.028        -4.358    -0.247
location_Huntsville         16.7330   4300.545      0.004      0.997     -8412.181  8445.647
location_Los+Angeles        -0.5306      0.399     -1.331      0.183        -1.312     0.250
location_New+York         9.064e-17      0.289   3.14e-16      1.000        -0.566     0.566
location_Philadelphia        0.1542      0.556      0.277      0.782        -0.936     1.245
location_Portland          -34.5689   1.61e+07  -2.15e-06      1.000     -3.15e+07  3.15e+07
location_San+Antonio       -34.5689   1.61e+07  -2.15e-06      1.000     -3.15e+07  3.15e+07
location_San+Diego          -0.6931      0.612     -1.132      0.258        -1.893     0.507
location_San+Francisco       0.9163      0.483      1.897      0.058        -0.030     1.863
location_Seattle             2.0541      0.475      4.324      0.000         1.123     2.985
location_St.+Louis          -1.3218      0.563     -2.349      0.019        -2.425    -0.219
location_Washington%C+DC     0.1603      0.284      0.565      0.572        -0.396     0.716
============================================================================================
"""

## Add Company And Job Title

In [97]:
categories = ["company", "categorizedTitle"]
for category in categories:
    series = data[category]
    dummies = pd.get_dummies(series, prefix=category)
    data = pd.concat([data, dummies], axis=1)
    #if(categories != "company"):
        #smdata = pd.concat([smdata, dummies], axis = 1)
    
#delete most common location for colinearity
#del data["location_"+mostCommonLocation]
del data["company_"+mostCommonCompany]
del data["categorizedTitle_"+str(mostCommonTitle)]

del data["Intercept"]

KeyError: 'Intercept'

### Normal


In [98]:
data.head()

,company,location,salary,title,HighLow,categorizedTitle,categorizedLevel,location_Atlanta,location_Austin,location_Charlotte,...,categorizedTitle_1,categorizedTitle_2,categorizedTitle_3,categorizedTitle_4,categorizedTitle_5,categorizedTitle_6,categorizedTitle_7,categorizedTitle_8,categorizedTitle_9,categorizedTitle_10
0,Texas A&M University,Houston,70000,Bioinformatics Postdoctoral Fellow,0,10,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Baylor College of Medicine,Houston,86500,Senior Staff Scientist / Technical Director,0,7,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Oscar Technology,Houston,160000,Big Data Analytics Manager,1,8,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Baylor College of Medicine,Houston,93431,Java Programmer,0,10,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Baylor College of Medicine,Houston,67632,Laboratory Manager,0,8,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [99]:
X = data.iloc[:,7:]
y = data["HighLow"]
y.name = "Actual"

In [100]:
logiReg = LogisticRegression()
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.905660377358


In [101]:
print "Accuracy score: ", metrics.accuracy_score(y,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score:  0.905660377358
Precision score:  0.911458333333
Recall score:  0.883838383838


In [102]:
pd.crosstab(y,predictions)


col_0,0,1
Actual,,
0,209,17
1,23,175


### Lasso

In [103]:
logiReg = LogisticRegression()
logiReg.penalty = "l1"
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.799528301887


In [104]:
print "Accuracy score: ", metrics.accuracy_score(y,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score:  0.799528301887
Precision score:  0.79274611399
Recall score:  0.772727272727


In [105]:
pd.crosstab(y,predictions)

col_0,0,1
Actual,,
0,186,40
1,45,153


### Ridge

In [106]:
logiReg = LogisticRegression()
logiReg.penalty = "l2"
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.905660377358


In [107]:
print "Accuracy score: ", metrics.accuracy_score(y,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score:  0.905660377358
Precision score:  0.911458333333
Recall score:  0.883838383838


In [108]:
pd.crosstab(y,predictions)

col_0,0,1
Actual,,
0,209,17
1,23,175


In [109]:
#np.exp(result.params).sort_values()
#conf = result.conf_int()
#conf["OR"]  = result.params
#conf.columns = ['2.5%','97.5','OR']
#np.exp(conf).sort_values('OR')

## Job Modifier
#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [125]:
data.head()

,company,location,salary,title,HighLow,categorizedTitle,categorizedLevel,location_Atlanta,location_Austin,location_Charlotte,...,categorizedTitle_5,categorizedTitle_6,categorizedTitle_7,categorizedTitle_8,categorizedTitle_9,categorizedTitle_10,categorizedLevel_0,categorizedLevel_1,categorizedLevel_2,categorizedLevel_3
0,Texas A&M University,Houston,70000,Bioinformatics Postdoctoral Fellow,0,10,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Baylor College of Medicine,Houston,86500,Senior Staff Scientist / Technical Director,0,7,1,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Oscar Technology,Houston,160000,Big Data Analytics Manager,1,8,4,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Baylor College of Medicine,Houston,93431,Java Programmer,0,10,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Baylor College of Medicine,Houston,67632,Laboratory Manager,0,8,4,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
levelData = data.groupby("categorizedLevel",as_index="False").count()
mostCommonLevel = levelData.idxmax()[1]
print "Most Common Categorized Title: ", mostCommonLevel


Most Common Categorized Title:  4


In [127]:
categories = ["categorizedLevel"]
for category in categories:
    series = data[category]
    dummies = pd.get_dummies(series, prefix=category)
    data = pd.concat([data, dummies], axis=1)

In [128]:
del data["categorizedLevel_"+str(mostCommonLevel)]
data

,company,location,salary,title,HighLow,categorizedTitle,categorizedLevel,location_Atlanta,location_Austin,location_Charlotte,...,categorizedTitle_9,categorizedTitle_10,categorizedLevel_0,categorizedLevel_1,categorizedLevel_2,categorizedLevel_3,categorizedLevel_0,categorizedLevel_1,categorizedLevel_2,categorizedLevel_3
0,Texas A&M University,Houston,70000,Bioinformatics Postdoctoral Fellow,0,10,4,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baylor College of Medicine,Houston,86500,Senior Staff Scientist / Technical Director,0,7,1,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Oscar Technology,Houston,160000,Big Data Analytics Manager,1,8,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Baylor College of Medicine,Houston,93431,Java Programmer,0,10,4,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Baylor College of Medicine,Houston,67632,Laboratory Manager,0,8,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,MD ANDERSON CANCER CENTER,Houston,64500,Clinical Research Scientist,0,9,4,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Baylor College of Medicine,Houston,54590,Senior Research Quality Assurance Analyst,0,6,1,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7,Baylor College of Medicine,Houston,43040,Research Quality Assurance Analyst,0,6,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Lighthouse Recruiting,Houston,67500,Sr. Certifying Scientist / Toxicologist,0,9,1,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,Baylor College of Medicine,Houston,32454,Cytometry Technician,0,10,4,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Normal


In [129]:
data.head()

,company,location,salary,title,HighLow,categorizedTitle,categorizedLevel,location_Atlanta,location_Austin,location_Charlotte,...,categorizedTitle_9,categorizedTitle_10,categorizedLevel_0,categorizedLevel_1,categorizedLevel_2,categorizedLevel_3,categorizedLevel_0,categorizedLevel_1,categorizedLevel_2,categorizedLevel_3
0,Texas A&M University,Houston,70000,Bioinformatics Postdoctoral Fellow,0,10,4,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baylor College of Medicine,Houston,86500,Senior Staff Scientist / Technical Director,0,7,1,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Oscar Technology,Houston,160000,Big Data Analytics Manager,1,8,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Baylor College of Medicine,Houston,93431,Java Programmer,0,10,4,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Baylor College of Medicine,Houston,67632,Laboratory Manager,0,8,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
X = data.iloc[:,7:]
y = data["HighLow"]
y.name = "Actual"

In [131]:
logiReg = LogisticRegression()
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.908018867925


In [132]:
print "Accuracy score: ", metrics.accuracy_score(y,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score:  0.908018867925
Precision score:  0.920634920635
Recall score:  0.878787878788


In [133]:
pd.crosstab(y,predictions)


col_0,0,1
Actual,,
0,211,15
1,24,174


### Lasso

In [134]:
logiReg = LogisticRegression()
logiReg.penalty = "l1"
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.799528301887


In [135]:
print "Accuracy score: ", metrics.accuracy_score(y,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score:  0.799528301887
Precision score:  0.79274611399
Recall score:  0.772727272727


In [136]:
pd.crosstab(y,predictions)

col_0,0,1
Actual,,
0,186,40
1,45,153


### Ridge

In [137]:
logiReg = LogisticRegression()
logiReg.penalty = "l2"
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.908018867925


In [138]:
print "Accuracy score: ", metrics.accuracy_score(y,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score:  0.908018867925
Precision score:  0.920634920635
Recall score:  0.878787878788


In [139]:
pd.crosstab(y,predictions)

col_0,0,1
Actual,,
0,211,15
1,24,174


#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [ ]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [ ]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [ ]:
## YOUR CODE HERE